## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [15]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from newsapi import NewsApiClient
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\orkhan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [16]:
# Read your api key environment variable
alpaca_api_key = os.getenv("ALPACA_API_KEY")

In [17]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=alpaca_api_key)

In [ ]:
bitcoin_news = newsapi.get_everything(q="jupyter lab", language="en")

In [7]:
# Fetch the Bitcoin news articles
btc_url = "https://news.bitcoin.com/bitcoin-ethereum-technical-analysis-bitcoin-moves-higher-as-crypto-markets-rebound-on-saturday/"

In [8]:
# Fetch the Ethereum news articles
eth_url = "https://news.bitcoin.com/bitcoin-ethereum-technical-analysis-bitcoin-moves-higher-as-crypto-markets-rebound-on-saturday/"

In [ ]:
# Create the Bitcoin sentiment scores DataFrame
# YOUR CODE HERE!

In [ ]:
# Create the Ethereum sentiment scores DataFrame
# YOUR CODE HERE!

In [ ]:
# Describe the Bitcoin Sentiment
# YOUR CODE HERE!

In [ ]:
# Describe the Ethereum Sentiment
# YOUR CODE HERE!

In [ ]:
### Questions:

Q: Which coin had the highest mean positive score?

A: 

Q: Which coin had the highest compound score?

A: 

Q. Which coin had the highest positive score?

A: 